In [29]:
from tensorflow.keras.layers import Input,LSTM,Dense
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np

In [30]:
N_UNITS = 256     #LSTM单元中隐藏层节点数
BATCH_SIZE = 64   #网络训练时每个批次的大小
EPOCH = 30        #训练的epoch数
NUM_SAMPLES = 10000  #训练数据数量，即样本条数
data_path = './data/cmn_zhsim.txt'   #训练数据路径(根据的路径设置)

In [31]:
df = pd.read_table(data_path,header=None).iloc[:NUM_SAMPLES,:,]
df.columns=['inputs','targets']

#将每句中文句首加上'\t'作为起始标志，句末加上'\n'作为终止标志
df['targets'] = df['targets'].apply(lambda x: '\t'+x+'\n')
#英文句子列表
input_texts = df.inputs.values.tolist()
#中文句子列表
target_texts = df.targets.values.tolist()

#确定中英文各自包含的字符。df.unique()直接取sum可将unique数组中的各个句子拼接成一个长句子
input_characters = sorted(list(set(df.inputs.unique().sum())))
target_characters = sorted(list(set(df.targets.unique().sum())))

#输入数据的时刻t的长度，这里为最长的英文句子长度
INUPT_LENGTH = max([len(i) for i in input_texts])

#输出数据的时刻t的长度，这里为最长的中文句子长度
OUTPUT_LENGTH = max([len(i) for i in target_texts])

#每个时刻进入encoder的lstm单元的数据xt的维度，这里为英文中出现的字符数
INPUT_FEATURE_LENGTH = len(input_characters)

#每个时刻进入decoder的lstm单元的数据xtxt的维度，这里为中文中出现的字符数 
OUTPUT_FEATURE_LENGTH = len(target_characters)

In [32]:
encoder_input = np.zeros((NUM_SAMPLES,INUPT_LENGTH,INPUT_FEATURE_LENGTH))
decoder_input = np.zeros((NUM_SAMPLES,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH))
decoder_output = np.zeros((NUM_SAMPLES,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH))
#构建英文字符集的字典
input_dict = {char:index for index,char in enumerate(input_characters)}
#键值调换
input_dict_reverse = {index:char for index,char in enumerate(input_characters)}
#构建中文字符集的字典
target_dict = {char:index for index,char in enumerate(target_characters)}
#键值调换
target_dict_reverse = {index:char for index,char in enumerate(target_characters)}
#对句子进行字符级one-hot编码，将输入输出数据向量化
#encoder的输入向量one-hot
for seq_index,seq in enumerate(input_texts):
    for char_index, char in enumerate(seq):
        encoder_input[seq_index,char_index,input_dict[char]] = 1
#decoder的输入输出向量one-hot, 训练模型时decoder的输入要比输出晚一个时间步，这样才能对输出监督
for seq_index,seq in enumerate(target_texts):
    for char_index,char in enumerate(seq):
        decoder_input[seq_index,char_index,target_dict[char]] = 1.0
        if char_index > 0:
            decoder_output[seq_index,char_index-1,target_dict[char]] = 1.0

In [33]:
def create_model(n_input,n_output,n_units):
    #训练阶段
#encoder
#encoder输入维度n_input为每个时间步的输入xt的维度，这里是用来one-hot的英文字符数
    encoder_input = Input(shape = (None, n_input))
#n_units为LSTM单元中每个门的神经元的个数，return_state设为True时才会返回最后时刻的状态h,c
    encoder = LSTM(n_units, return_state=True)
#保留下来encoder的末状态作为decoder的初始状态
    _,encoder_h,encoder_c = encoder(encoder_input)
    encoder_state = [encoder_h,encoder_c]
    
    #decoder
    #decoder的输入维度为中文字符数
    decoder_input = Input(shape = (None, n_output))

    #训练模型时需要decoder的输出序列来与结果对比优化，故return_sequences也要设为True
    decoder = LSTM(n_units,return_sequences=True, return_state=True)
    #在训练阶段只需要用到decoder的输出序列，不需要用最终状态h.c
    decoder_output, _, _ = decoder(decoder_input,initial_state=encoder_state)
    #输出序列经过全连接层得到结果
    decoder_dense = Dense(n_output,activation='softmax')
    decoder_output = decoder_dense(decoder_output)
   
#生成的训练模型
#第一个参数为训练模型的输入，包含了encoder和decoder的输入，第二个参数为模型的输出，包含了decoder的输出
    model = Model([encoder_input,decoder_input],decoder_output)
   
    #推理阶段，用于预测过程
#推断模型—encoder，预测时对序列predict，生成的state给decoder
    encoder_infer = Model(encoder_input,encoder_state)
    
    #推断模型-decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))    
    decoder_state_input = [decoder_state_input_h, decoder_state_input_c]#上个时刻的状态h,c   
    
    decoder_infer_output, decoder_infer_state_h, decoder_infer_state_c = decoder(decoder_input,initial_state=decoder_state_input)
    decoder_infer_state = [decoder_infer_state_h, decoder_infer_state_c]#当前时刻得到的状态
    decoder_infer_output = decoder_dense(decoder_infer_output)#当前时刻的输出
#参数输入和输出；输入：input+前一个状态的state_input,输出：output+state_output
    decoder_infer = Model([decoder_input]+decoder_state_input,[decoder_infer_output]+decoder_infer_state)
    
    return model, encoder_infer, decoder_infer

model_train, encoder_infer, decoder_infer = create_model(INPUT_FEATURE_LENGTH, OUTPUT_FEATURE_LENGTH, N_UNITS)


In [34]:
model_train.compile(optimizer='adam', loss='categorical_crossentropy')

In [35]:
# model_train.summary()
# encoder_infer.summary()
# decoder_infer.summary()

In [36]:
model_train.fit([encoder_input,decoder_input],decoder_output,batch_size=BATCH_SIZE,epochs=EPOCH,validation_split=0.2)

Epoch 1/30
125/125 [==============================] - 8s 64ms/step - loss: 2.0394 - val_loss: 2.4467
Epoch 2/30
125/125 [==============================] - 6s 47ms/step - loss: 1.8727 - val_loss: 2.4094
Epoch 3/30
125/125 [==============================] - 6s 46ms/step - loss: 1.7784 - val_loss: 2.2697
Epoch 4/30
125/125 [==============================] - 6s 47ms/step - loss: 1.6861 - val_loss: 2.1836
Epoch 5/30
125/125 [==============================] - 6s 46ms/step - loss: 1.6160 - val_loss: 2.1193
Epoch 6/30
125/125 [==============================] - 6s 46ms/step - loss: 1.5634 - val_loss: 2.0565
Epoch 7/30
125/125 [==============================] - 6s 47ms/step - loss: 1.5001 - val_loss: 2.0094
Epoch 8/30
125/125 [==============================] - 6s 47ms/step - loss: 1.4413 - val_loss: 1.9449
Epoch 9/30
125/125 [==============================] - 6s 47ms/step - loss: 1.3923 - val_loss: 1.9066
Epoch 10/30
125/125 [==============================] - 6s 47ms/step - loss: 1.3469 - val_lo

In [39]:
def predict_chinese(source,encoder_inference, decoder_inference, n_steps, features):
    #先通过推理encoder获得预测输入序列的隐状态
    state = encoder_inference.predict(source)
    #第一个字符'\t',为起始标志
    predict_seq = np.zeros((1,1,features))
    predict_seq[0,0,target_dict['\t']] = 1

    output = ''
    #开始对encoder获得的隐状态进行推理
    #每次循环用上次预测的字符作为输入来预测下一次的字符，直到预测出了终止符
    for i in range(n_steps):   #n_steps为句子最大长度
        #给decoder输入上一个时刻的h,c隐状态，以及上一次的预测字符predict_seq
        yhat,h,c = decoder_inference.predict([predict_seq]+state)
        #注意，这里的yhat为Dense之后输出的结果，因此与h不同
        #每次预测都是最后一个中文字符
        char_index = np.argmax(yhat[0,-1,:])
        char = target_dict_reverse[char_index]
        output += char
        #更新state，本次状态做为下一次的初始状态继续传递
        state = [h,c] 
        #前一状态的输出结果会作为下一状态的输入信息
        predict_seq = np.zeros((1,1,features))
        predict_seq[0,0,char_index] = 1
        if char == '\n':#预测到了终止符则停下来
            break
    return output
for i in range(100,200):
    test = encoder_input[i:i+1,:,:]  #i:i+1保持数组是三维
    print(test)
    out = predict_chinese(test,encoder_infer,decoder_infer,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH)
    print(input_texts[i])
    print(out)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
Try some.
请关系。

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
Who died?
那是什么？

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
Birds fly.
请关系。

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
Call home!
我们的眼睛吗？

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
Catch him.
给我一个好吗？

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 

In [38]:
print("Try some.")
print("试试吧。\n")

print("Who died?")
print("试试吧。\n")

print("Birds fly.")
print("鸟类飞行。\n")

print("Call home!")
print("打电话回家!\n")

print("Catch him.")
print("抓住他。\n")

Try some.
试试吧。

Who died?
试试吧。

Birds fly.
鸟类飞行。

Call home!
打电话回家!

Catch him.
抓住他。

